<a href="https://colab.research.google.com/github/Marvimnas/Distance_Calculation/blob/main/distancias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
clientes = [
    {'Local': 'SJC1', 'latitude': -23.2215, 'longitude': -45.9101},
    {'Local': 'SJC2', 'latitude': -23.2005, 'longitude': -45.9172},
    {'Local': 'SJC3', 'latitude': -23.1673, 'longitude': -45.8863},
    {'Local': 'SJC4', 'latitude': -23.1667, 'longitude': -45.7992},
    {'Local': 'SJC5', 'latitude': -23.2147, 'longitude': -45.8007},
    {'Local': 'SJC6', 'latitude': -23.2347, 'longitude': -45.8747},
    {'Local': 'SJC7', 'latitude': -23.1977, 'longitude': -45.9287}
]

In [ ]:
data = pd.DataFrame.from_dict(clientes)
data.head()

,Local,latitude,longitude
0,SJC1,-23.2215,-45.9101
1,SJC2,-23.2005,-45.9172
2,SJC3,-23.1673,-45.8863
3,SJC4,-23.1667,-45.7992
4,SJC5,-23.2147,-45.8007


In [ ]:
lat_fabrica = -23.2021
long_fabrica = -45.8582

In [ ]:
from geopy.distance import geodesic
import requests

In [ ]:
for index, row in data.iterrows():
    coordenadas_fabrica = (lat_fabrica, long_fabrica)
    coordenadas_clientes = (row['latitude'], row['longitude'])
    distancia = round(geodesic(coordenadas_fabrica, coordenadas_clientes).kilometers,2)
    print(f"Distância da fabrica para {row['Local']}: {distancia} km")

Distância da fabrica para SJC1: 5.73 km
Distância da fabrica para SJC2: 6.04 km
Distância da fabrica para SJC3: 4.81 km
Distância da fabrica para SJC4: 7.2 km
Distância da fabrica para SJC5: 6.05 km
Distância da fabrica para SJC6: 3.99 km
Distância da fabrica para SJC7: 7.23 km


In [ ]:
def calculate_road_distance(origin, destination):
    url = "http://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix"
    params = {
        "origins": f"{origin[0]},{origin[1]}",
        "destinations": f"{destination[0]},{destination[1]}",
        "travelMode": "driving",
        "key": "AmQ4bukp9WLfN9GqIntfn9pJugIvSSSfM7OxMxws5Up0F88YqlsXAA6IAPk6_vdI"
    }
    resposta = requests.get(url, params=params)
    data = resposta.json()
    if "resourceSets" in data and data["resourceSets"]:
        resultado = data["resourceSets"][0]["resources"]
        if resultado:
            return resultado[0]["results"][0]["travelDistance"]
    return None

for index, row in data.iterrows():
    coordenadas_clientes = (row['latitude'], row['longitude'])
    distancia_real = round(calculate_road_distance((lat_fabrica, long_fabrica), coordenadas_clientes),2)
    if distancia_real is not None:
        print(f"Distância para {row['Local']}: {distancia_real} km")
    else:
        print(f"Não foi possível calcular a distância para {row['Local']}")

Distância para SJC1: 9.4 km
Distância para SJC2: 13.07 km
Distância para SJC3: 7.93 km
Distância para SJC4: 9.89 km
Distância para SJC5: 14.33 km
Distância para SJC6: 11.67 km
Distância para SJC7: 13.96 km


In [ ]:
!pip install folium
import folium

In [ ]:
mapa = folium.Map(location=[lat_fabrica, long_fabrica], zoom_start=12)

for index, row in data.iterrows():
    coordenadas_clientes = [row['latitude'], row['longitude']]
    distancia = calculate_road_distance((lat_fabrica, long_fabrica), coordenadas_clientes)
    if distancia is not None:
        folium.Marker(location=coordenadas_clientes, popup=f"{row['Local']} ({distancia} km)").add_to(mapa)

folium.Marker(location=[lat_fabrica, long_fabrica], popup='Fabrica', icon=folium.Icon(color='red')).add_to(mapa)

mapa.save("map.html")
mapa